In [47]:
import pandas as pd
import requests
from nltk.tokenize import word_tokenize

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re 
from string import punctuation 
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
#import pyLDAvis.sklearn
import nltk
nltk.download('stopwords')
import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud, STOPWORDS

for dependency in (
    "brown",
    "names",
    "wordnet",
    "averaged_perceptron_tagger",
    "universal_tagset",
):
    nltk.download(dependency)


import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to

In [48]:
r = requests.get('https://newsapi.org/v2/everything?q=tesla&from=2024-12-29&sortBy=publishedAt&apiKey=5ba4bef2c78d47b4b31d19dbf0c0fd50')
x = r.json()
articles = x.get("articles", [])
data = []
for article in articles:
    data.append({
        "author": article.get("author"),
        "title": article.get("title"),
        "publishedAt": article.get("publishedAt"),
        "content": article.get("content"),
        "source_name": article.get("source", {}).get("name") 
    })
df = pd.DataFrame(data)

df

,author,title,publishedAt,content,source_name
0,Peter Johnson,"Honda wants to launch an EV for under $30,000 ...",2025-01-28T19:18:26Z,"A new Honda EV for under $30,000? That’s the p...",Electrek
1,The Associated Press,Elon Musk’s X partners with Visa on payment se...,2025-01-28T19:18:26Z,NEW YORK (AP) X is teaming up with Visa to soo...,Financial Post
2,"Derek Saul, Forbes Staff, \n Derek Saul, Forbe...",DeepSeek Panic Live Updates: Nvidia Stock Rebo...,2025-01-28T19:13:58Z,The release of a less capital-intensive artifi...,Forbes
3,Jörn Brien,Streit der Tech-Titanen: Bill Gates kritisiert...,2025-01-28T19:01:34Z,In seiner im September 2023 erschienenen Biogr...,T3n
4,,"Microsoft, TikTok'u satın almayı planlıyor",2025-01-28T19:00:00Z,"a').click(); event.preventDefault();""&gt;Tam B...",Donanimhaber.com
...,...,...,...,...,...
91,Vítor M.,Surpresa: um em cada quatro Tesla Model 3 não ...,2025-01-28T16:00:13Z,O Tesla Model 3 está a ter problemas. Esta sit...,Sapo.pt
92,Wille Wilhelmsson,Tv-serie om Happy Face på ingång,2025-01-28T16:00:00Z,34 idag\r\nVälkommen till Feber\r\nSkaffa Febe...,Feber.se
93,Yiğit Gençer,OpenAI CEO’su Sam Altman’dan DeepSeek için açı...,2025-01-28T16:00:00Z,"Çin merkezli DeepSeek, dün itibariyle tüm NVID...",Shiftdelete.net
94,Fred Lambert,Tesla announces all V4 Superchargers can now c...,2025-01-28T15:55:45Z,Tesla has announced all V4 Superchargers can n...,Electrek


In [49]:
stop_words = set(stopwords.words('english'))
def clean_data(text,remove_stop_words=True,lemmatize_words=True):
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"ur", " your ", text)
    text = re.sub(r" nd "," and ",text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" tkts "," tickets ",text)
    text = re.sub(r" c "," can ",text)
    text = re.sub(r" e g ", " eg ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) 
    text = re.sub(r" u "," you ",text)
    text = re.sub(r" em "," them ",text)
    text = re.sub(r" re "," are ",text)
    text = re.sub(r" s "," is ",text)
    text = re.sub(r" m "," am ",text)
    text = re.sub(r" b "," be ",text)
    text = re.sub(r" d "," and ",text)
    text = re.sub(r" u "," you ",text)
    text = re.sub(r" ur "," your ",text)
    text = re.sub(r" nd "," and ",text)
    text=text.lower()
    text=''.join([i for i in text if i not in punctuation])
    if remove_stop_words:
        text = ' '.join([word for word in text.split() if word not in stop_words])
    if lemmatize_words:
        lemmatizer = WordNetLemmatizer()
        text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])    
    return text
    

In [50]:
df["clean_content"]=df["content"].apply(lambda x: clean_data(x))
df["clean_title"]=df["title"].apply(lambda x: clean_data(x))
df["clean_author"]=df["author"].apply(lambda x: clean_data(str(x)))

In [51]:
df["clean_source_name"]=df["source_name"].apply(lambda x: clean_data(str(x)))

In [52]:
df

,author,title,publishedAt,content,source_name,clean_content,clean_title,clean_author,clean_source_name
0,Peter Johnson,"Honda wants to launch an EV for under $30,000 ...",2025-01-28T19:18:26Z,"A new Honda EV for under $30,000? That’s the p...",Electrek,new honda ev 30000 that’s plan could come soon...,honda want launch ev 30in u early next year,peter johnson,electrek
1,The Associated Press,Elon Musk’s X partners with Visa on payment se...,2025-01-28T19:18:26Z,NEW YORK (AP) X is teaming up with Visa to soo...,Financial Post,new york ap x teaming visa soon offer system r...,elon musk’s x partner visa payment service eff...,associated press,financial post
2,"Derek Saul, Forbes Staff, \n Derek Saul, Forbe...",DeepSeek Panic Live Updates: Nvidia Stock Rebo...,2025-01-28T19:13:58Z,The release of a less capital-intensive artifi...,Forbes,release less capitalintensive artificial intel...,deepseek panic live update nvidia stock reboun...,derek saul forbes staff derek saul forbes staf...,forbes
3,Jörn Brien,Streit der Tech-Titanen: Bill Gates kritisiert...,2025-01-28T19:01:34Z,In seiner im September 2023 erschienenen Biogr...,T3n,seiner im september erschienenen biografie bes...,streit der techtitanen bill gate kritisiert el...,jörn brien,t3n
4,,"Microsoft, TikTok'u satın almayı planlıyor",2025-01-28T19:00:00Z,"a').click(); event.preventDefault();""&gt;Tam B...",Donanimhaber.com,aclick eventpreventdefaultgttam boyutta görabd...,microsoft tiktoku satın almayı planlıyor,,donanimhabercom
...,...,...,...,...,...,...,...,...,...
91,Vítor M.,Surpresa: um em cada quatro Tesla Model 3 não ...,2025-01-28T16:00:13Z,O Tesla Model 3 está a ter problemas. Esta sit...,Sapo.pt,tesla model está ter problemas esta situação e...,presa um cada quatro tesla model não passa na ...,vítor,sapopt
92,Wille Wilhelmsson,Tv-serie om Happy Face på ingång,2025-01-28T16:00:00Z,34 idag\r\nVälkommen till Feber\r\nSkaffa Febe...,Feber.se,idag välkommen till feber skaffa feber regerin...,tvserie om happy face på ingång,wille wilhelmsson,feberse
93,Yiğit Gençer,OpenAI CEO’su Sam Altman’dan DeepSeek için açı...,2025-01-28T16:00:00Z,"Çin merkezli DeepSeek, dün itibariyle tüm NVID...",Shiftdelete.net,çin merkezli deepseek dün itibariyle tüm nvidi...,openai ceo’su sam altman’dan deepseek için açı...,yiğit gençer,shiftdeletenet
94,Fred Lambert,Tesla announces all V4 Superchargers can now c...,2025-01-28T15:55:45Z,Tesla has announced all V4 Superchargers can n...,Electrek,tesla announced v4 supercharger charge 325kw n...,tesla announces v4 supercharger charge 325kw n...,fred lambert,electrek


In [53]:
vectorization=TfidfVectorizer(max_df=0.95,min_df=2,max_features=1000,stop_words='english')
vectorization.fit(df["clean_content"])
X=vectorization.transform(df["clean_content"])
feature_names=vectorization.get_feature_names_out()
print(feature_names)


['2025' '28' '31' '371' '385' '617' 'access' 'actually' 'ai' 'al'
 'amerikanischen' 'annon' 'announced' 'ante' 'ap' 'appeared' 'apple'
 'artificial' 'att' 'au' 'auch' 'auf' 'autobauer' 'autohersteller'
 'automatiquement' 'aux' 'bewogen' 'bi' 'biden' 'bien' 'billionaire' 'bmw'
 'borne' 'brancher' 'brott' 'business' 'campaign' 'capitalintensive'
 'capitolio' 'car' 'ce' 'ceo' 'cette' 'char' 'charge' 'charging' 'chez'
 'chill' 'china' 'chinese' 'chino' 'ciencia' 'city' 'comercio' 'comisión'
 'company' 'confirmed' 'content' 'cut' 'cybertruck' 'da' 'dazu' 'decline'
 'deepseek' 'del' 'dem' 'demand' 'den' 'der' 'det' 'deutschen'
 'deutschland' 'die' 'dnes' 'donald' 'du' 'duffy' 'dör' 'earnings'
 'eautos' 'el' 'electric' 'elektrofahr' 'elon' 'email' 'en' 'ene' 'enero'
 'esta' 'estados' 'este' 'está' 'et' 'eu' 'europa' 'eusonderzölle'
 'euzölle' 'ev' 'exrepresentante' 'feber' 'film' 'finally' 'financial'
 'fix' 'fonctionnalité' 'founder' 'från' 'fullyear' 'fund' 'föreslår'
 'förstärkare' 'gate' 

In [54]:
X=X.toarray()
X=pd.DataFrame(X,columns=vectorization.get_feature_names_out())
X

,2025,28,31,371,385,617,access,actually,ai,al,...,wehren,year,yearslong,york,zeit,är,äußerungen,år,åte,électrique
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.236671,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.335751,0.297435,0.0,0.0,0.0,0.00000,0.00000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.334205,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.19059,0.19059,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0


In [55]:
num_topics=3
lda=LatentDirichletAllocation(n_components=num_topics,learning_method='online',random_state=42,max_iter=1)
lda.fit(X)

LatentDirichletAllocation(learning_method='online', max_iter=1, n_components=3,
                          random_state=42)

In [58]:
from langdetect import detect
df["language"]=df["content"].apply(lambda x: detect(x) if x is not None else None)


In [56]:
def print_topics(model, feature_names, num_top_words):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))


print_topics(lda, feature_names,5)        

Topic 0:
tesla ing hors char la
Topic 1:
deepseek ha et que su
Topic 2:
char tesla annon välkommen till


In [65]:
from gensim.models import Word2Vec
wv=Word2Vec(df["clean_content"].apply(lambda x: x.split()),min_count=1)
word_vector = wv.wv["tesla"]
word_vector


array([-0.00032638,  0.00386553, -0.00643476, -0.00146919,  0.00788574,
        0.00608988, -0.00350356,  0.00419076, -0.00851223,  0.00586325,
       -0.0049351 , -0.00342131,  0.00911583,  0.00115235,  0.00764948,
       -0.00655614,  0.00525363,  0.00939201, -0.00871622, -0.00625379,
       -0.00680883, -0.00462986, -0.00362243, -0.00900368,  0.00759188,
       -0.00484656,  0.00832241,  0.00472114, -0.00707675,  0.0044013 ,
        0.00622439, -0.00725062, -0.00706259, -0.00311372, -0.00883573,
       -0.00063985, -0.00024519,  0.00252268,  0.0007366 , -0.00159915,
       -0.00547365,  0.00140349, -0.00123788,  0.00678566,  0.00472427,
        0.00401778,  0.00154489, -0.00284799, -0.0040963 , -0.00055749,
        0.00182403, -0.00297505, -0.00773593, -0.00781724, -0.00983872,
       -0.00553798, -0.00180233, -0.00454947, -0.00699291, -0.00378224,
        0.00468963, -0.00343583,  0.00831961,  0.00144546, -0.00812118,
        0.01004869,  0.00771155,  0.00549159, -0.00756823,  0.00

In [71]:
import numpy as np

def document_vector(doc):
    words = [word for word in doc if word in wv.wv]
    if len(words) == 0:
        return np.zeros(100)  # في حالة عدم وجود كلمات مفهومة
    return np.mean([wv.wv[word] for word in words], axis=0)

# تمثيل كل مستند بعد التنظيف
doc_vectors = np.array([document_vector(doc) for doc in df['clean_content']])


In [72]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

pca = PCA(n_components=10, random_state=42)
doc_vectors_pca = pca.fit_transform(doc_vectors)

scaler = MinMaxScaler()
doc_vectors_scaled = scaler.fit_transform(doc_vectors_pca)

lda = LatentDirichletAllocation(n_components=5, random_state=42)
topics = lda.fit_transform(doc_vectors_scaled)


In [74]:
def print_topics(model, feature_names, num_top_words):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))


print_topics(lda, feature_names,5)        

Topic 0:
617 385 access actually ai
Topic 1:
actually ai al 371 617
Topic 2:
actually ai al 371 617
Topic 3:
actually ai al 371 617
Topic 4:
617 access 385 28 actually
